In [ ]:
import pandas as pd
import numpy as np
import sys, gc, os

In [ ]:
os.getcwd()

In [ ]:
df_discordant = pd.read_csv("https://raw.githubusercontent.com/ELELAB/LUAD_LUSC_TCGA_comparison/master/6-recount/LUSC/discordant_samples.txt", header=None)
df_tcgabiolinks = pd.read_csv("../TCGA_biolinks.csv", index_col=0).dropna(how="all", axis=1)
df_tcgabiolinks = df_tcgabiolinks[df_tcgabiolinks["cancer.type"].isin(["LUAD","LUSC"])].set_index("pan.samplesID")
df_tcgabiolinks

In [ ]:
df_files=pd.read_csv("files_tcga.dat", index_col=0)
df_files=df_files.drop_duplicates("cases.0.submitter_id")
df_files.sort_values('cases.0.submitter_id', inplace=True)

bins = np.linspace(0,5,num=4)
cutted = pd.cut(df_files["cases.0.samples.0.longest_dimension"].replace("unknown", np.nan).dropna().astype(float),bins=bins, labels=bins[1:])
df_files["longest_dimension"]=df_files["cases.0.samples.0.longest_dimension"]
df_files["longest_dimension"][cutted.index]=cutted

df_files=df_files[df_files['cases.0.submitter_id'].isin(df_tcgabiolinks.index)]

In [ ]:
pd.read_csv("../TCGA_biolinks.csv", index_col=0)["cancer.type"].unique()

In [ ]:
for (sample,case) in zip(df_files.index,df_files["cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id"]):
    if case in df_discordant.values.ravel():
        df_files.at[sample,"cancer.type"]="Discordant_LUSC"

In [ ]:
df_files = df_files.reset_index().set_index('cases.0.submitter_id')
df_files = df_files.transpose().append(df_tcgabiolinks.reindex(index=df_files.index).transpose()).transpose()

In [ ]:
df_files.dropna(how="any", axis=1, thresh=len(df_files.index)/10, inplace=True)
df_files.reset_index().set_index("file_name").fillna("unknown").to_csv("files.dat", index=True)

## mainTable

In [ ]:
df_files=df_files.fillna("unknown").set_index("file_name")
df_files.to_csv("files.dat", index=True, header=True)

In [ ]:
df = pd.read_csv("mainTable_all.csv",index_col=0)

In [ ]:
df.columns[~df.columns.isin(df_files.index)]

In [ ]:
df = df.reindex(columns=df.columns[df.columns.isin(df_files.index)])
df.to_csv("mainTable.csv", index=True, header=True)
df_files.fillna("unknown").to_csv("files.dat", index=True)

In [ ]:
import scanpy as sc

In [ ]:
adata = sc.AnnData(X=df.transpose(), obs=df_files.reindex(index=df.columns))
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000)
sc.pl.highly_variable_genes(adata)
hvg = adata.var[adata.var["highly_variable"]==True].index

In [ ]:
df.reindex(index=hvg).to_csv("mainTable.csv", index=True, header=True)

### sbmtm

In [ ]:
from sbmtm import sbmtm

In [ ]:
hsbm = sbmtm()

In [ ]:
hsbm.make_graph_from_BoW_df(df.reindex(index=hvg))

In [ ]:
hsbm.save_graph("graph_hv.xml.gz")

In [ ]:
del hsbm
gc.collect()

# Merged

In [ ]:
df_merged = pd.read_csv("../../merged/files.dat", index_col=0)
df_query = pd.read_csv("../../merged/files.txt",sep="\t", index_col=4)

In [ ]:
df_merged["Subtype"]=df_merged["status"]
df_merged["Type"]=df_merged["status"]
df_tmp = df_files.reset_index().set_index("cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id")
df_tmp=df_tmp[df_tmp.index.isin(df_merged.index)]
for sample in df_tmp.index:
    df_merged.at[sample,"Type"]=df_tmp.at[sample,"cancer.type"]
    df_merged.at[sample,"Subtype"]=df_tmp.at[sample,"Subtype_Selected"]
for sample in df_merged.index:
    if sample[:12] in df_query.index and df_merged.at[sample,"status"]=="tumor":
        df_merged.at[sample,"Type"]=df_query.at[sample[:12],"project.project_id"][-4:]
    if sample in df_discordant.values:
        df_merged.at[sample,"Type"]="Discordant_LUSC"

In [ ]:
df_merged=df_merged[df_merged["primary_site"]=="Lung"]
df_merged.fillna("unknown").to_csv("../../merged/lung/files.dat")